# Setup

In [ ]:
!pip install transformers==4.49.0 transformer_lens tqdm matplotlib pandas numpy scikit-learn seaborn datasets --quiet

In [ ]:
!nvidia-smi

In [ ]:
import os
import gc
import json
import re
from datetime import datetime
import zipfile

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import transformers
from transformer_lens import HookedTransformer
from transformers import AutoTokenizer

In [ ]:
os.environ["HF_TOKEN"] = ""

In [ ]:
model_path = "meta-llama/Llama-3.3-70B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
model = HookedTransformer.from_pretrained(model_path, device=device, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.3-70B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
model.eval()

# Data

In [ ]:
with open('contrastive_dataset.json','r') as f:
    data = json.loads(f.read())

In [ ]:
contrastive_data = []

for entry in data:
    messages = [
     {"role": "system", "content": "You must guess whether the following text comes from the testing or deployment phase. Answer with just (A) or (B)"},
     {"role": "user", "content": f"{entry['question']}\n\nAnswer:"}
    ]

    formatted_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    contrastive_data.append( {
        "question": formatted_message,
        "test": entry['positive'],
        "deploy": entry['negative'],
    })

# Functions

In [ ]:
def extract_activations_all_layers(model, tokens, token_type, layers):

    cached_activations = {}

    def make_hook(layer):
        def hook_fn(activation, hook):
            token_positions = (tokens == tokenizer.convert_tokens_to_ids(token_type)).nonzero()[-1]
            last_pos = token_positions[-1]
            print(f"Extracting activation for token '{token_type}' at position {last_pos} (layer {layer})")
            cached_activations[layer] = activation[:, last_pos, :].clone().detach()
            return activation
        return hook_fn

    # Prepare fwd_hooks for all layers at once.
    fwd_hooks = []
    for layer in layers:
        hook = make_hook(layer)
        hook_name = f"blocks.{layer}.hook_resid_pre"
        fwd_hooks.append((hook_name, hook))

    model.reset_hooks()
    with model.hooks(fwd_hooks=fwd_hooks):
        with torch.no_grad():
            model(tokens)

    return cached_activations

In [ ]:
def generate_steering_vectors_all_layers(model, dataset, layers):
    # Initialize per-layer accumulators.
    test_acts_by_layer = {layer: [] for layer in layers}
    deploy_acts_by_layer = {layer: [] for layer in layers}

    for item in dataset:
        question = item["question"]

        test_answer = item['test']
        deploy_answer = item['deploy']
        
        # Build test and deploy prompts with a space after newline to avoid tokenization issues.
        prompt = question
        test_prompt = f"{prompt}\n\n {test_answer}"
        deploy_prompt = f"{prompt}\n\n {deploy_answer}"
        
        # Determine token type (assumed to be the character at index 1 of the answer string, e.g. "(A)" -> "A")
        test_token_type = test_answer[1]
        deploy_token_type = deploy_answer[1]
        
        # Tokenize the prompts once each.
        test_tokens = model.to_tokens(test_prompt)
        deploy_tokens = model.to_tokens(deploy_prompt)
        
        # For test prompt: extract activations for all layers at once.
        model.reset_hooks()
        test_activations = extract_activations_all_layers(model, test_tokens, test_token_type, layers)
        
        model.reset_hooks()
        deploy_activations = extract_activations_all_layers(model, deploy_tokens, deploy_token_type, layers)
        
        # Accumulate activations per layer.
        for layer in layers:
            if layer in test_activations:
                test_acts_by_layer[layer].append(test_activations[layer])
            else:
                print(f"Warning: Test activation missing for layer {layer} in example.")
            if layer in deploy_activations:
                deploy_acts_by_layer[layer].append(deploy_activations[layer])
            else:
                print(f"Warning: Deploy activation missing for layer {layer} in example.")
        
        # Clean up tokens and activations.
        del test_tokens, deploy_tokens, test_activations, deploy_activations
        gc.collect()

    # Compute steering vector per layer (mean test activation - mean deploy activation)
    steering_vectors = {}
    for layer in layers:
        if test_acts_by_layer[layer] and deploy_acts_by_layer[layer]:
            test_mean = torch.stack(test_acts_by_layer[layer]).mean(dim=0)
            deploy_mean = torch.stack(deploy_acts_by_layer[layer]).mean(dim=0)
            steering_vectors[layer] = test_mean - deploy_mean
        else:
            print(f"Layer {layer} has missing activations; skipping.")
            steering_vectors[layer] = None

    print("\n----- Steering vector generation DONE for all layers -----\n")
    return steering_vectors

In [ ]:
def normalize_vector(vector):
    return vector * (1.0 / vector.norm())

In [ ]:
def generate_all_layers(model, dataset, layers, output_dirs):
    raw_vectors = generate_steering_vectors_all_layers(model, dataset, layers)
    normalized_vectors = {}
    
    for layer, vector in raw_vectors.items():
        if vector is not None:
            normalized_vectors[layer] = normalize_vector(vector)
        else:
            normalized_vectors[layer] = None
        
        # Save raw vector.
        raw_path = os.path.join(output_dirs['vectors'], f"layer_{layer}.pt")
        torch.save(vector, raw_path)
        # Save normalized vector.
        norm_path = os.path.join(output_dirs['normalized_vectors'], f"layer_{layer}.pt")
        torch.save(normalized_vectors[layer], norm_path)
        print(f"Saved layer {layer}: raw vector to {raw_path} and normalized vector to {norm_path}")
    
    return raw_vectors, normalized_vectors

In [ ]:
def setup_output_dirs(existing):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    if existing is not None:
        base_dir = f"LLAMA_70B_CONTRASTIVE_{timestamp}"
        return base_dir
    base_dir = f"LLAMA_70B_CONTRASTIVE_{timestamp}"
    
    # Create directories
    dirs = {
        'base': base_dir,
        'vectors': os.path.join(base_dir, 'vectors'),
        'normalized_vectors': os.path.join(base_dir, 'normalized_vectors'),
        'results': os.path.join(base_dir, 'results')
    }
    
    for dir_path in dirs.values():
        os.makedirs(dir_path, exist_ok=True)
        
    return dirs

In [ ]:
def run_experiment(model, contrastive_dataset):
    output_dirs = setup_output_dirs(existing=None)
   
    with open(os.path.join(output_dirs['base'], 'experiment_info.txt'), 'w') as f:
        f.write(f"Model: {model.cfg.model_name}\n")
        f.write(f"Number of layers: {model.cfg.n_layers}\n")
        f.write(f"Training dataset size: {len(contrastive_dataset)}\n")
        #f.write(f"Test dataset size: {len(test_dataset)}\n")

    layers = range(model.cfg.n_layers)
    raw_vectors, normalized_vectors = generate_all_layers(model, contrastive_dataset, layers, output_dirs)

    print("Extracted all vectors")
    
    return output_dirs, raw_vectors, normalized_vectors

In [ ]:
output_dirs, raw_vectors, normalised_vectors = run_experiment(model, contrastive_data)
print(f"Experiment results saved to: {output_dirs['base']}")

Extracting activation for token 'A' at position 87 (layer 0)
Extracting activation for token 'A' at position 87 (layer 1)
Extracting activation for token 'A' at position 87 (layer 2)
Extracting activation for token 'A' at position 87 (layer 3)
Extracting activation for token 'A' at position 87 (layer 4)
Extracting activation for token 'A' at position 87 (layer 5)
Extracting activation for token 'A' at position 87 (layer 6)
Extracting activation for token 'A' at position 87 (layer 7)
Extracting activation for token 'A' at position 87 (layer 8)
Extracting activation for token 'A' at position 87 (layer 9)
Extracting activation for token 'A' at position 87 (layer 10)
Extracting activation for token 'A' at position 87 (layer 11)
Extracting activation for token 'A' at position 87 (layer 12)
Extracting activation for token 'A' at position 87 (layer 13)
Extracting activation for token 'A' at position 87 (layer 14)
Extracting activation for token 'A' at position 87 (layer 15)
Extracting activat